In [1]:
# Imports Training Data
import sys
sys.path.append('../lambda/data')

from tweet_ner_data_label import train_data

In [2]:
# Imports & Dependencies
from __future__ import unicode_literals, print_function

import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


In [3]:
print(spacy.__version__)

2.3.5


In [4]:
# Update original data to include Normal NER Tags to prevent forgetting problem
nlp = spacy.load('en_core_web_md')

In [5]:
# Checks entities if multiple truck locations
def count_locations(ents_arr):
    count = 0
    
    for i in range(0,len(ents_arr)):
        cur_ent_type = ents_arr[i][2]
        if cur_ent_type == 'TRUCK_LOCATION':
            count += 1
    
    return count

#MODIFY THIS
# NO OVERLAPS WITH TRUCK DATA
    
# Adds old NER tags to train data
def append_old_ner(train_data):
    for i in range(0,len(train_data)):
        doc = nlp(train_data[i][0])
        ents_arr = train_data[i][1]['entities']
        locations_count = count_locations(ents_arr)

        for ent in doc.ents:
            start = int(ent.start_char)
            end = int(ent.end_char)
            
            # Cases
            # < Start---End > Bound entire word and more
            case1 = lambda truck_start, truck_end: start <= truck_start and end >= truck_end
            # < Start>---End< Bound entire left up to end
            case2 = lambda truck_start, truck_end: start <= truck_start and end <= truck_end and end >= truck_start
            # > Start---<End > Bound entire right up to start
            case3 = lambda truck_start, truck_end: end >= truck_end and start >= truck_start and start <= truck_end
            # Bound within word
            case4 = lambda truck_start, truck_end: start >= truck_start and end <= truck_end
            
            # If no TRUCK_LOCATION ent then just add all entities
            if locations_count == 0:
                ents_arr.append((start, end, ent.label_))
            else:
                continue_outer = False
                for j in range(0,locations_count):
                    truck_start = int(ents_arr[j][0])
                    truck_end = int(ents_arr[j][1])

                    if case1(truck_start, truck_end) or case2(truck_start, truck_end) or case3(truck_start, truck_end) or case4(truck_start, truck_end):
                        continue_outer = True
                        break
                    
                if continue_outer:
                    continue
                else:
                    ents_arr.append((start, end, ent.label_))
                    
    return train_data

train_data = append_old_ner(train_data)

## For testing individual tweets
# output = []
# output = append_old_ner([train_data[7]])
# print(output)


In [6]:
# Validate old NER Tags
print(train_data[105])

('#tour de fat today. Come down to Yards park and try your favorite Cuban speciality paired with New Belgium beer.', {'entities': [(33, 43, 'TRUCK_LOCATION'), (0, 12, 'PERSON'), (13, 18, 'DATE'), (66, 71, 'NORP'), (95, 106, 'GPE')]})


In [7]:
# Load model if exists
def load_model(model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    return nlp

# List of Names Entities
# To be referenced later on
move_names = []

# Train update Spacy's NER
def train_NER(train_data, iterations, model=None):
    TRAIN_DATA = train_data
    
    # Set language to only Text in tweets
    nlp = load_model(model)
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')
        
    # add new labels eg: using TRUCK_LOCATION
    for _, annotations in TRAIN_DATA:
        print(_)
        print(annotations)
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)
            
    return nlp

In [8]:
# Train Data
nlp = train_NER(train_data, 5)

Created blank 'en' model
'This is my f**king country': Racist white woman arrested for attacking passengers on New York bus https://t.co/Rfm0jauh72
{'entities': [(12, 19, 'PERSON'), (86, 94, 'GPE')]}
RT @mussie671: Freestyle #3, Fiasco type shiiii https://t.co/0GTJscWORW
{'entities': [(0, 2, 'ORG'), (25, 27, 'CARDINAL'), (29, 35, 'GPE'), (41, 47, 'PERSON')]}
What you think Cole? @JColeNC @JColeDirect https://t.co/oE4Ipvq6hL
{'entities': [(15, 19, 'PRODUCT')]}
⚡️ “Redskins dominate the Raiders on Sunday Night Football ”https://t.co/4VuHWOs1gX
{'entities': [(4, 12, 'NORP'), (26, 33, 'ORG'), (37, 58, 'WORK_OF_ART')]}
RT @NWSSanJuan: 215PM FLASH FLOOD EMERGENCY for A Dam Failure in Isabela Municipality y Quebradillas Municipality in Puerto Rico... #prwx h…
{'entities': [(0, 2, 'ORG'), (65, 113, 'GPE'), (117, 128, 'GPE'), (132, 139, 'PERSON')]}
Posted @chinatown on the corner of 7th &amp; D st. NW, PERFECT WEATHER TO SLIDE UP!!
{'entities': [(25, 53, 'TRUCK_LOCATION')]}
Good Morning DMV, we

Truck 1 is open at K &amp; 14th Streets NW. Franklin Square. By the motorcycle parking. Stop by for a warm slice of app… https://t.co/KkKSDTKdc1
{'entities': [(19, 42, 'TRUCK_LOCATION'), (6, 7, 'CARDINAL'), (44, 59, 'FAC')]}
Truck 1 has your lunch on board at Franklin Square today. 14th &amp; K NW. We're away from the other trucks near the mo… https://t.co/njmsQ0REr9
{'entities': [(35, 50, 'TRUCK_LOCATION'), (58, 73, 'TRUCK_LOCATION'), (51, 56, 'DATE')]}
Truck 1 is open at 1st &amp; Patterson NE. Take a look at the menu and swing by for a slice. https://t.co/xu6aqkVfzq
{'entities': [(19, 41, 'TRUCK_LOCATION'), (6, 7, 'CARDINAL')]}
Truck 1 is at NoMa this morning. Setting up for an 11am opening time. Stand by for a peek at today's menu.
{'entities': [(14, 18, 'TRUCK_LOCATION'), (6, 7, 'CARDINAL'), (51, 53, 'CARDINAL'), (93, 98, 'DATE')]}
Truck 1 is open at NoMa First and Patterson NE. Guinness Steak Pie is on the truck in very limited supply. Perfect… https://t.co/Iv0kiDn1W1
{'entities'

{'entities': [(25, 37, 'TRUCK_LOCATION'), (37, 56, 'TRUCK_LOCATION'), (0, 5, 'DATE')]}
Good morning Van Ness kabob_king is here and window will open at 11.
{'entities': [(13, 21, 'TRUCK_LOCATION'), (5, 12, 'TIME'), (65, 67, 'CARDINAL')]}
Good morning friendship heights kabob_king is here and window will open at 11.
{'entities': [(13, 31, 'TRUCK_LOCATION'), (5, 12, 'TIME'), (32, 42, 'PERSON'), (75, 77, 'CARDINAL')]}
Good morning CNN kabob_king is here and window will open at 11.
{'entities': [(13, 16, 'TRUCK_LOCATION'), (5, 12, 'TIME'), (17, 27, 'PERSON'), (60, 62, 'CARDINAL')]}
Good morning Van Ness kabob_king is here and window will open at 11.
{'entities': [(13, 21, 'TRUCK_LOCATION'), (5, 12, 'TIME'), (65, 67, 'CARDINAL')]}
Good morning CNN kabob_king is here and window will open at 11.
{'entities': [(13, 16, 'TRUCK_LOCATION'), (5, 12, 'TIME'), (17, 27, 'PERSON'), (60, 62, 'CARDINAL')]}
Good morning Van Ness kabob_king is here and window will open at 11.
{'entities': [(13, 21, 'TRUCK

We’re open TODAY AT EASTERN MARKET  635 NORTH CAROLINA AVE SE FROM 10:30-3:30pm CANT WAIT TO SEE YOU!!! https://t.co/W1TQTEA8VT
{'entities': [(20, 34, 'TRUCK_LOCATION'), (36, 61, 'TRUCK_LOCATION'), (11, 16, 'DATE'), (67, 79, 'TIME')]}
Thank you for having us! https://t.co/SugfIznkJI
{'entities': []}
Good morning Rosslyn! We are are next to 1001 Wilson Blvd until 2:00pm. See you all soon
{'entities': [(13, 20, 'TRUCK_LOCATION'), (41, 57, 'TRUCK_LOCATION'), (5, 12, 'TIME')]}
RT @FairWindsBrew: Two new beers on tap, @quickwraps onsite and @Charliebrews giving brewery tours at 3, 5pm #vacraftbeer #drinklocal https…
{'entities': [(0, 2, 'ORG'), (19, 22, 'CARDINAL'), (102, 103, 'CARDINAL'), (105, 106, 'QUANTITY'), (122, 133, 'PERSON')]}
RT @StoddardDC1902: Quick Wraps food truck is at Stoddard Baptist Global Care I ordered a Veggie Wrap #quickwraps  @quickwraps https://t.co…
{'entities': [(0, 2, 'ORG'), (49, 77, 'ORG'), (90, 101, 'ORG'), (102, 115, 'MONEY'), (115, 139, 'ORG')]}
Good morning 

/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  **kwargs
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  **kwargs
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Saté1. Lenfant Plaza / 600 Maryland Ave SWSaté2. U..." with entities "[(7, 20, 'TRUCK_LOCATION'), (23, 42, 'TRUCK_LOCATI...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the ali

...

/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Its Friday and today we will be serving lunch here..." with entities "[(53, 83, 'TRUCK_LOCATION'), (4, 10, 'DATE'), (15,...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Come get some #dinner we'll be in #Alexandria from..." with entities "[(35, 45, 'TRUCK_LOCATION'), (59, 92, 'TRUCK_LOCAT...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/farrisism

/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "SAFFRON FOOD LOVERS AT GEORGETOWN TODAY. SAFFRON 2..." with entities "[(23, 54, 'TRUCK_LOCATION'), (0, 19, 'ORG')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Get her what she really wants today!  We’re open @..." with entities "[(50, 63, 'TRUCK_LOCATION'), (30, 35, 'DATE'), (68...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/farrisismati/anaco

/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "The Mesob at Metro Center #vegan serving the Best ..." with entities "[(13, 24, 'TRUCK_LOCATION'), (41, 64, 'ORG')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Join us for lunch today at Metro Center (Near 12th..." with entities "[(27, 39, 'TRUCK_LOCATION'), (46, 66, 'TRUCK_LOCAT...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/farrisismati/anac

/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "It’s a little windy and chilly, but beautiful skie..." with entities "[(90, 105, 'TRUCK_LOCATION'), (73, 85, 'TIME')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "RT @truckeroo: Tomorrow is the last Truckeroo of t..." with entities "[(4, 13, 'TRUCK_LOCATION'), (15, 23, 'DATE'), (36,...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/farrisismati/an

Losses {'ner': 12672.59254726299}
Losses {'ner': 9418.857374320834}
Losses {'ner': 8198.82800791554}
Losses {'ner': 7203.751590738799}
Losses {'ner': 6512.93336649704}


In [9]:
# Review NER Results
def show_ents(doc):
    doc = nlp(doc[0])
    if doc.ents:
        for ent in doc.ents:
            print(ent.text + ' - ' + ent.label_)
    else:
        print('No entities found')

In [11]:
# Test across subset of tweets
# for i in range(0,10):
#     show_ents(train_data[i])
#     print("")
    
print(train_data[75])
print('')
show_ents(train_data[75])

('L’Enfant! It’s almost lunch time! 11:30-2pm we will be cooking for you! https://t.co/g4SXEUqM05', {'entities': [(0, 8, 'TRUCK_LOCATION'), (34, 43, 'TIME')]})

L’Enfant - TRUCK_LOCATION
11:30-2pm - TIME


In [15]:
# Manually test the trained model
def test_model(nlp, test_text):
    doc = nlp(test_text)
    print("Entities in '%s' \n" % test_text )
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
test_model(nlp, "Find us at faragut square today")

Entities in 'Find us at faragut square today' 

TRUCK_LOCATION faragut square
DATE today


In [25]:
# Save model to output directory
def save_model(nlp, output_dir, name):
    if output_dir is not None:
        output_dir = Path(output_dir)
        
        if not output_dir.exists():
            output_dir.mkdir()
            
        nlp.meta[name] = name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

def test_saved_model(model_dir, test_text):
    # test the saved model
    print("Loading from", model_dir)
    nlp = spacy.load(model_dir)

    # Check the classes have loaded back consistently
    # assert nlp.get_pipe("ner").move_names == move_names
    pipe_found = False
    for pipe in nlp.get_pipe('ner').move_names:
        if pipe == 'B-TRUCK_LOCATION':
            pipe_found = True
            break
            
    if pipe_found == False:
        print('NER Pipe doesn\'t have Truck Location')
    
    test_model(nlp, test_text)

In [17]:
save_model(nlp,'../lambda/model/tweet_ner_model','ner_1')

Saved model to ../lambda/model/tweet_ner_model


In [26]:
test_saved_model('../lambda/model/tweet_ner_model', "Today we are at 2nd Ave Parrish")

Loading from ../lambda/model/tweet_ner_model
Entities in 'Today we are at 2nd Ave Parrish' 

DATE Today
TRUCK_LOCATION 2nd Ave Parrish


In [1]:
# Imports & Dependencies
from __future__ import unicode_literals, print_function

from numpy import load

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight

import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

# Data
from tweet_data_label import train_data

# Spacy Load
nlp = spacy.load('en')

/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/farris

693
Tweet 1
'This is my f**king country': Racist white woman arrested for attacking passengers on New York bus https://t.co/Rfm0jauh72
_____

Tweet 2
RT @mussie671: Freestyle #3, Fiasco type shiiii https://t.co/0GTJscWORW
_____

Tweet 3
What you think Cole? @JColeNC @JColeDirect https://t.co/oE4Ipvq6hL
_____

Tweet 4
⚡️ “Redskins dominate the Raiders on Sunday Night Football ”https://t.co/4VuHWOs1gX
DATE: Sunday Night 
_____

Tweet 5
RT @NWSSanJuan: 215PM FLASH FLOOD EMERGENCY for A Dam Failure in Isabela Municipality y Quebradillas Municipality in Puerto Rico... #prwx h…
NEIGHBORHOOD: Isabela Municipality NEIGHBORHOOD: Quebradillas Municipality 
TIME: 215PM 
_____

Tweet 6
Posted @chinatown on the corner of 7th &amp; D st. NW, PERFECT WEATHER TO SLIDE UP!!
ADDRESS: 7th &amp; D st. NW 

POSITIVE: Posted @chinatown on the corner of 
_____

Tweet 7
Good Morning DMV, we are @ChinaTown on the corner of 7th &amp; G st NW. Hand crafted burgers, endless toppings, Hand cut fries,  PULL UP !!!



Tweet 310
RT @truckeroo: Tomorrow is the last Truckeroo of the season! Come out to celebrate the start of an amazing weekend. See you there!Trucker…

NEGATIVE
_____

Tweet 311
We have missed ALL our fans in #Tysons speared the news we are there for lunch tomorrow #vegan #Egyptian #yummy… https://t.co/uviH5VhAL1

NEGATIVE
_____

Tweet 312
Good morning! Yellow Tin heads towards to US ICE today. See you all during lunch hour.

POSITIVE
_____

Tweet 313
Good morning Navy Yardian! Yellow Tin landed in front of Five Guys. See you all during lunch hour.

POSITIVE
_____

Tweet 314
Good morning! Yellow Tin heads towards to Navy Yard today. See you all during lunch hour.

POSITIVE
_____

Tweet 315
Good morning! Yellow Tin heads towards to Metro Center today. See you all during lunch hour.

POSITIVE
_____

Tweet 316
Good morning! Yellow Tin heads towards to US ICE today. See you all during lunch hour.

POSITIVE
_____

Tweet 317
Good morning! Yellow Tin landed at Farragut Square to serve lunch to

_____

Tweet 595
Monday we will be at Union station with fresh Bibim bap, Bul goki with Jap chae.  See you~

NEGATIVE
_____

Tweet 596
Tuesday we will be at Union station with fresh Korean food, Bibim bap, Bul goki with Jap chae. See you tomorrow at lunch time.

NEGATIVE
_____

Tweet 597
Thursday we will be at 500 12th St. SW in front of ICE with fresh Korean food for your lunch. Stay warm and see you tomorrow.

NEGATIVE
_____

Tweet 598
Good morning Clarendon! Unfortunately we can't make it today, parking is full.

NEGATIVE
_____

Tweet 599
Franklin square Thursday. Can't wait to serve fresh Korean food to you then. See you~

NEGATIVE
_____

Tweet 600
Union station Satuday. Can't find parking today. Open window at 11 tomorrow. See you~

NEGATIVE
_____

Tweet 601
Friday we will be at L'Enfant plaza with fresh and delicious Korean food. You need some hot and spicy food for this cold week. See you~

NEGATIVE
_____

Tweet 602
Thursday we will be at Franklin square with fresh Bibim bap, Bu

In [22]:
class DataTokenizer:
    def __init__(self, train_data):
        self.tokenizer = Tokenizer()
        self.train_data = train_data

        # Sets up tokenizer
        try:
            self.__init_tokenizer()
        except ModuleNotFoundError:
            raise

    # Clean data
    def __tweet_clean(self, text):
        lower = []
        text = text.replace('&amp;', 'and')

        for token in nlp(text):
            lower.append(token.text.lower())

        return lower
            
    # Orginize Tweet Data for tokenizer
    def __organize_tweet_data(self):
        tweet_sequences = []

        # Organize tweets & train data into arrays
        for tweet_data in self.train_data:
            tweet_sequences.append(self.__tweet_clean(tweet_data[0]))

        return tweet_sequences
    
    def __init_tokenizer(self):
        data = self.__organize_tweet_data()
        self.tokenizer.fit_on_texts(data)
        
    # Tokenization Methods
    def tokenize_tweets(self, data):
        self.tokenizer.fit_on_texts(data)

        return self.tokenizer.texts_to_sequences(data)

class NERModel:
    def __init__(self, ner_model_path):
        try:
            self.nlp_ner = spacy.load(ner_model_path)
        except ModuleNotFoundError:
            raise
            
    def predict(self, text):
        # Check the classes have loaded back consistently
        # assert nlp.get_pipe("ner").move_names == move_names
        if self.nlp_ner.get_pipe('ner').move_names[0] != 'B-TRUCK_LOCATION':
            print('NER Pipe doesn\'t have Truck Location')
            return
        
        doc = self.nlp_ner(text)
        
        if doc.ents:
            return doc.ents
        else:
            return []
    
class AffirmationModel:
    def __init__(self, affirmation_model_path, scaler_data_path, train_data, max_len):
        self.max_len = max_len # Max Tweet Length
        
        try:
            self.scaler_data = self.__load_scaler_data(scaler_data_path, 'scaler_data.npy')
        except ModuleNotFoundError:
            raise
        
        # Sets up tokenizer
        try:
            # Setup Tokenizer
            self.DataTokenizer = DataTokenizer(train_data)
            self.tokenizer = self.DataTokenizer.tokenizer
        except ModuleNotFoundError:
            raise
            
        try:
            self.nlp_affirmation = keras.models.load_model(affirmation_model_path, compile=False)
        except ModuleNotFoundError:
            raise
            
    def predict(self,tweet):
        # Predict affirmation
        tokenized_tweets = self.DataTokenizer.tokenize_tweets([self.__tweet_lower(tweet)])
        predict_tweets = [self.__pad_array(data, self.max_len) for data in tokenized_tweets]
        scaled_test_data = self.__scale_test_data(predict_tweets)
        affirmation_prediction = self.nlp_affirmation.predict_classes(scaled_test_data)
        
        if affirmation_prediction[0] == 0:
            return True
        else:
            return False
    
    # Tweet lower
    def __tweet_lower(self, text):
        text = text.replace('&amp;', 'and')

        return text.lower()
        
    def __create_zeros_array(self, length):
        zeros_arr = []

        i = 0
        while i < length:
            zeros_arr.append(0)
            i += 1

        return zeros_arr

    def __pad_array(self,data, max_len):
        zeros_len = self.max_len - len(data)
        zeros_arr = self.__create_zeros_array(zeros_len)

        return [*data,*zeros_arr]
    
    def __load_scaler_data(self, directory, file):
        try:
            path = f'{directory}/{file}'
            data = load(path)
            
            return data
        except ModuleNotFoundError:
            raise
            
    def __scale_test_data(self, data):
        scaler_object = MinMaxScaler()
        scaler_object.fit(self.scaler_data)
        scale_data = scaler_object.transform(data)
        
        return scale_data

class AssertModel:
    def __init__(self, affirmationModel, nerModel):
        self.affirmationModel = affirmationModel
        self.nerModel = nerModel

    def predict(self, tweet):
        affirmation = self.affirmationModel.predict(tweet)
        named_entities = self.nerModel.predict(tweet)
 
        location_types = {
            'TRUCK_LOCATION': [],
            'FAC': [],
            'GPE': [],
            'LOC': [],
        }
        
        for ent in named_entities:
            if ent.label_ in location_types:
                location_types[ent.label_].append(ent.text)
        
        result = {
            'tweet': tweet,
            'affirmation': affirmation,
            'locations': location_types
        }
        
        return result

In [23]:
affirmationModel = AffirmationModel('./tweet_affirmation_model/test', './tweet_affirmation_model', train_data, 35)
nerModel = NERModel('./tweet_ner_model')
assertModel = AssertModel(affirmationModel, nerModel)

NameError: name 'load' is not defined

In [60]:
print(assertModel.predict("Today we are at 5th street"))

{'tweet': 'Today we are at 5th street', 'affirmation': True, 'locations': {'TRUCK_LOCATION': ['5th street'], 'FAC': [], 'GPE': [], 'LOC': []}}


In [ ]:
# Next steps, move & improve this into a properly formatted structure for AWS Lambda